In [1]:
import os

In [5]:
os.getcwd()

'/Volumes/JK/Swizzl/swizzl/services'

In [2]:
from bs4 import BeautifulSoup
import requests 
from mlearning import sentiment as st
from mlearning import prof
import re
import html

/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.21.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 0.21.0. This might lead to breaking code or invalid results. Use at your own ri

In [27]:
def linkImg(link):
    url = "{}".format(link)
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,"lxml") 
    mydivs = soup.find("div", class_="caas-img-container")
    return mydivs

In [28]:
print(linkImg("https://news.yahoo.com/italy-miss-hitler-among-19-160352734.html"))

None


In [2]:
def linkText(link):
    url = "{}".format(link)
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,"lxml") 
    article_text = ''
    mylink = soup.find('div').findAll('p')
    for i in mylink:
        article_text += ''.join(i.findAll(text = True))
    return article_text

In [4]:
def YahooFetch():
    url = "https://news.yahoo.com/rss/"
    url = requests.get(url)
    soup = BeautifulSoup(url.text,'xml')
    
    try:
        # Fetch All Required items
        titles = soup.findAll('title')
        links = soup.findAll('link')
        pubDates = soup.findAll('pubDate')

    except Exception as e :
        
        # Return Empty if titles, links, descriptions, pubdates not found
        print(e)
        return "Error"

    # We don't want the first elements as these are just metadata
    
    titles.pop(0)
    titles.pop(0)
    links.pop(0)
    links.pop(0)
    pubDates.pop(0)

    #Dictionary to hold crawled information

    FeedDict = {}
    temp = []
    id = []
    
    try:
        # Add titles to the Dictionary
            
        
        for i in titles:
            temp.append(html.unescape(i.get_text()))
            FeedDict['title'] = temp
            
        # Add link and details regarding text contetn @ link to the Dictionary
        
        temp = []
        temptext = []
        tbscore = []
        vadscore = []
        profvalue = []
        for i in links:
            
            # Append Links
            temp.append(html.unescape(i.get_text()))
            
            # Append Text Content from Links
            textval = linkText(html.unescape(i.get_text()))
            textval = re.sub('\"','\\"',textval)
            textval = " \" " + textval + " \" "
            temptext.append(textval)
            
            # Find Subjectivity, Objectivity of text content
            score = st.sentimentTB(textval)
            tbscore.append(score)
            
            # Find Sentiment of text content
            score = st.sentimentVader(textval)
            vadscore.append(score)
            
            # Find Profanity Score of content
            textval = [textval]
            score = float(prof.predProf(textval))
            profvalue.append(score)
            
            # Add to Feeds Dictionary 
            FeedDict['link'] = temp
            FeedDict['linktext'] = temptext
            FeedDict['tbScore'] = tbscore
            FeedDict['vaderScore'] = vadscore
            FeedDict['prof'] = profvalue
            
        # Add Published Dates to the Dictionary
        temp = []
        for i in pubDates:
            temp.append(i.get_text())
            FeedDict['pubdate'] = temp
        
        print("Success")
    except Exception as e :
        print(e)
        return "Error"
    
    return FeedDict

In [5]:
FeedDict = YahooFetch()

Success


In [6]:
FeedDict

{'title': ['Kushner named Trump’s border-wall czar — along with practically everything else in government',
  'Then and now: Swiss glacier photos show impact of global warming',
  'German museum confirms 49-carat diamond among heist haul',
  "A New Wildfire Is Threatening Southern California at Thanksgiving. Here's What to Know",
  'ICE arrested an estimated 250 people who enrolled in a fake university set up by federal authorities as part of an immigration sting operation',
  'Russia says it showed nuclear missile system to U.S. inspectors',
  "No excuse for silence on China's camps for Uighurs: exiled leader",
  'Utah banning ‘conversion therapy’ with Mormon church backing',
  '21 of the Most Beautiful Sacred Sites That Every Traveler Must Visit',
  "Trump peddles 'war on Thanksgiving' that he probably heard about on Fox News",
  'Republican running against Ilhan Omar banned from Twitter after calling for rival to be hanged',
  'First bodies of UK truck victims arrive in Vietnam',
  